In [85]:
from langchain_community.vectorstores import Pinecone
from langchain_openai import OpenAIEmbeddings
from langchain_community.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.chains import RetrievalQAWithSourcesChain
from langchain import prompts

# import SystemMessagePromptTemplate
from langchain.prompts import SystemMessagePromptTemplate

import pinecone
import os

In [90]:
import datetime
import json
import openai
import os
import pandas as pd
import pinecone
import re
from tqdm.auto import tqdm
from typing import List, Union
import zipfile

# Langchain imports
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import BaseChatPromptTemplate, ChatPromptTemplate
from langchain import SerpAPIWrapper, LLMChain
from langchain.schema import AgentAction, AgentFinish, HumanMessage, SystemMessage
# LLM wrapper
from langchain.chat_models import ChatOpenAI
from langchain import OpenAI
# Conversational memory
from langchain.memory import ConversationBufferWindowMemory

In [9]:
# get the api keys
open_ai_key = os.getenv('OPENAI_API_KEY') or 'OPENAI_API_KEY'

# init embeddings model
embed = OpenAIEmbeddings(
    model='text-embedding-ada-002',
)

In [10]:
# init vector store
PINECONE_API_KEY = os.getenv('PINECONE_RAGDEMO_API_KEY') or 'PINECONE_RAGDEMO_API_KEY'
PINECONE_ENVIRONMENT = os.getenv('PINECONE_RAGDEMO_ENV') or 'PINECONE_RAGDEMO_ENV'

pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENVIRONMENT
)

In [12]:
text_field = "text"
index_name = 'rag-demo'

# create index if it does not exist
if index_name not in pinecone.list_indexes():
    # we create a new index
    pinecone.create_index(
        name=index_name,
        metric='cosine',
        dimension=1536  # 1536 dim of text-embedding-ada-002
    )

# switch back to normal index for langchain
index = pinecone.Index(index_name)

In [13]:

vectorstore = Pinecone(
    index, embed, text_field
)

vector_db_index = pinecone.GRPCIndex(index_name)

# completion llm
llm_model = ChatOpenAI(
    model_name='gpt-3.5-turbo',
    temperature=0.0
)

qa_chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm_model,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

c:\Users\paulr\AppData\Local\pypoetry\Cache\virtualenvs\rag-demo-ggeqyuoK-py3.12\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [56]:
from langchain.agents import load_tools

tools = load_tools(
    ['llm-math'],
    llm=llm_model
)

In [ ]:
from langchain.chains import RetrievalQA

retrieval_llm = OpenAI(temperature=0)

podcast_retriever = RetrievalQA.from_chain_type(llm=retrieval_llm, chain_type="stuff", retriever=docsearch.as_retriever())

In [57]:
from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    vectorstore.as_retriever(),
    "Astrophysics-Knowledge-Base",
    "Searches and returns information Astrophysics.",
)
tools.append(tool)
tools

[Tool(name='Calculator', description='Useful for when you need to answer questions about math.', func=<bound method Chain.run of LLMMathChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question'], template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question}\n'), llm=ChatOpenAI(client=<openai.resources.chat.completions.Comp

In [89]:
from langchain.agents import AgentExecutor, create_openai_tools_agent

from langchain import hub

# prompt = hub.pull("hwchase17/openai-tools-agent")
prompt = prompts. ChatPromptTemplate.from_messages(
    [
        ("system", "You know everything about Astrophysics. Only answer qustions by consulting your knowledge base. If you don't know the answer, say 'I don't know'."),
        ("human", "{input}")
        
    ]
)

agent = create_openai_tools_agent(llm_model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, return_intermediate_steps=True)

ValueError: Prompt missing required variables: {'agent_scratchpad'}

In [84]:
# custom_prompt: SystemMessagePromptTemplate = SystemMessagePromptTemplate.from_template("Your custom system message here")
# # prompt[0] = custom_prompt
# prompt[0].Config.system_message = custom_prompt
# prompt[0]

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant'))

In [ ]:
# CustomOpenAIFunctionsAgent.create_prompt(system_message=SystemMessage(content=prefix))

In [88]:
chat_history = []
# result = agent_executor.invoke({"input": "3^3.5"})
result = agent_executor.invoke({"input": "explain the collapse of a dense core"})
# result = agent_executor.invoke({"input": "explain unicorns"})



> Entering new AgentExecutor chain...

Invoking: `Astrophysics-Paper-Retriever` with `{'query': 'collapse of a dense core'}`


[Document(page_content='clouds (see Kainulainen et al. 2009 ), and it has been previously\nshown in simulations that it results naturally from such a fragmen-\ntation cascade (see e.g. Federrath et al. 2010b ;S c h m a l z le ta l . 2010 ;\nBallesteros-Paredes et al. 2011 ; Kritsuk, Norman & Wagner 2011 ;\nVeltchev, Klessen & Clark 2011 ).\n2.4.1 Turbulent density ﬁelds in a collapsing background\nIn the case of collapsing protostellar cores, the density evolution\nis inﬂuenced by the gravitational collapse which pumps energy\ninto turbulence, potentially leading to large density ﬂuctuations and\nfurther fragmentation of the cloud. Hopkins ( 2013a ) developed a\nsimple model for collapsing spherical clouds which assumes a con-\nstant virial parameter (based on Robertson & Goldreich 2012 and\nMurray & Chang 2015 ).7Of course, a perfectly spherical collapse\nwo

In [63]:
result

{'input': 'search the database and explain the collapse of a dense core',
 'output': 'The collapse of a dense core refers to the process by which a dense region within a molecular cloud collapses under its own gravity, leading to the formation of a protostar. This phenomenon is of great interest in astrophysics as it is a crucial step in the formation of stars.\n\nThe collapse of a dense core is a complex process that involves several physical mechanisms. One important factor is turbulence within the molecular cloud, which can lead to the fragmentation of the cloud into smaller dense cores. These smaller cores then undergo gravitational collapse, with the densest regions becoming even denser and eventually forming protostars.\n\nThe collapse of a dense core is influenced by the interplay between gravity and turbulence. As the core collapses, gravitational energy is converted into turbulent energy, which can lead to large density fluctuations and further fragmentation of the cloud. The 